In [1]:
from absbox import API,Generic

In [2]:
localAPI = API("https://absbox.org/api/latest",lang='english',check=False)

⠼ Connecting engine server -> https://absbox.org/api/latest

✅Connected, local lib:0.20.2, server:0.23.2

In [3]:
from absbox import API

asset = ["AdjustRateMortgage"
        ,{"originBalance":73_875.00
          ,"originRate":["floater",0.04,{"index":"USCMT1Y"
                                        ,"spread":0.01
                                        ,"reset":"YearFirst"}]
          ,"originTerm":360
          ,"freq":"Monthly","type":"Level","originDate":"1999-05-01"
          ,"arm":{"initPeriod":2,"firstCap":0.01,"periodicCap":0.01,"lifeCap":0.09}}
          ,{"currentBalance":20_788.41
          ,"currentRate":0.0215
          ,"remainTerm":77
          ,"status":"current"}]

GNMA_36208ALG4 = Generic(
    "820146/36208ALG4/G2-Custom AR"
    ,{"collect":["2023-05-01","2023-05-31"]
        ,"pay":["2023-05-26","2023-06-28"]
        ,"stated":"2070-01-01"
        ,"poolFreq":"MonthEnd"
        ,"payFreq":["DayOfMonth",20]}
    ,{'assets':[asset]
     ,'issuanceStat':{"IssuanceBalance":10000}
     }
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":20_899.37
             ,"rate":0.025
             ,"originBalance":1_553_836.00
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"floater": [0.07,"USCMT1Y",0.01,"YearFirst"],"dayCount":"DC_30_360_US"}
             ,"bondType":{"Sequential":None}
             ,"lastAccrueDate":"2023-04-30"})
      ,)
    ,(("Ginnie_Mae_guaranty",
       {"type":{"annualPctFee":[("poolBalance",),0.0006]}
       ,"feeDueDate":"2023-04-26"}),
      ("service_fee",
       {"type":{"annualPctFee":[("poolBalance",)
                                ,("Max"
                                  ,("substract",("poolWaRate",),("bondRate","A1"))
                                  ,("constant",0))]}
       ,"feeDueDate":"2023-04-26"}))
    ,{"amortizing":[
         ["calcFee","Ginnie_Mae_guaranty","service_fee"]
         ,["payFee","acc01",['Ginnie_Mae_guaranty',"service_fee"]]
         ,["payInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]]
      ,"endOfCollection":[
          ["liqSupport","Ginnie_Mae","account","acc01"
              ,{"formula": 
                ("floorWithZero"
                ,("substract",("cumPoolDefaultedBalance",)
                            ,("liqBalance","Ginnie_Mae")))}
              ]
          ,["calcInt","A1"]]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,{"Ginnie_Mae":{"type":"Unlimited","start":"2023-05-26"}}
    ,None)


r = localAPI.run(GNMA_36208ALG4
                ,runAssump = [("inspect",("MonthEnd",("cumPoolDefaultedBalance",))
                                         ,("MonthEnd",("liqBalance","Ginnie_Mae")))
                              ,("interest",("USCMT1Y",0.0468))]
                ,poolAssump = ("Pool"
                              ,("Mortgage",{"CDR":0.005},None,{"Rate":0.3,"Lag":4},None)
                              ,None
                              ,None)
                ,read=True)

# Inspect cumulative defaulted balance
r['result']['inspect']['<CumulativePoolDefaultedBalance>']

# Inspect credit provided by Ginnie Mae
r['result']['inspect']['<LiqBalance:Ginnie_Mae>']

# the cash deposited to SPV in account `acc01`
r['accounts']['acc01'][r['accounts']['acc01']["memo"]=="<Support:Ginnie_Mae>"]

⠇

,balance,change,memo
date,,,
2023-05-31,331.21,8.56,<Support:Ginnie_Mae>
2023-06-30,331.25,8.74,<Support:Ginnie_Mae>
2023-07-31,330.75,8.37,<Support:Ginnie_Mae>
2023-08-31,330.78,8.54,<Support:Ginnie_Mae>
2023-09-30,333.11,8.44,<Support:Ginnie_Mae>
...,...,...,...
2029-05-31,314.08,0.63,<Support:Ginnie_Mae>
2029-06-30,313.80,0.52,<Support:Ginnie_Mae>
2029-07-31,313.46,0.38,<Support:Ginnie_Mae>


In [4]:
r['accounts']['acc01'].loc['2023-06-30']

,balance,change,memo
date,,,
2023-06-30,0.00,0.00,<PoolInflow:CollectedRecoveries>
2023-06-30,0.00,0.00,<PoolInflow:CollectedPrepayment>
2023-06-30,225.27,225.27,<PoolInflow:CollectedPrincipal>
2023-06-30,322.51,97.24,<PoolInflow:CollectedInterest>
2023-06-30,331.25,8.74,<Support:Ginnie_Mae>


In [5]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2023-05-01,20788.41,0.00,0.00,0,0.00,0.00,0.00,0.0000,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-31,20555.55,224.30,98.35,0,8.56,0.00,0.00,0.0568,None,None,224.30,0.0,0.0,8.56,0.00,0.00
2023-06-30,20321.54,225.27,97.24,0,8.74,0.00,0.00,0.0568,None,None,449.57,0.0,0.0,17.30,0.00,0.00
2023-07-31,20086.93,226.24,96.14,0,8.37,0.00,0.00,0.0568,None,None,675.81,0.0,0.0,25.67,0.00,0.00
2023-08-31,19851.18,227.21,95.03,0,8.54,0.00,0.00,0.0568,None,None,903.02,0.0,0.0,34.21,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029-05-31,1237.73,305.80,7.30,0,0.63,0.35,0.81,0.0568,None,None,19196.99,0.0,0.0,353.69,104.88,244.92
2029-06-30,930.09,307.12,5.85,0,0.52,0.31,0.72,0.0568,None,None,19504.11,0.0,0.0,354.21,105.19,245.64
2029-07-31,621.27,308.44,4.40,0,0.38,0.24,0.57,0.0568,None,None,19812.55,0.0,0.0,354.59,105.43,246.21
